### Word2Vec Implementation from Scratch

# Introduction
 Word2Vec is a popular technique for word embeddings, which captures the meaning of words by placing them in a continuous vector space.
 In this exercise, you will implement Word2Vec using NumPy and complete the missing parts of the code.
We will represent each word as a one-hot vector, meaning each word in the vocabulary is mapped to a unique binary vector with only one active (1) position.

import necessary libraries

In [1]:
import numpy as np
from collections import defaultdict
from numpy.linalg import norm

In [2]:
### Adjust the hyperparameters if needed ###
settings = {
	'window_size': 2,
	'n': 10,
	'epochs': 50,
	'learning_rate': 0.01
}

In [85]:
class word2vec:
    def __init__(self, settings):
        """
        Initialize the Word2Vec model with given hyperparameters.
        """
        ## Start code
        self.n = settings['n']
        self.lr = settings['learning_rate']
        self.epochs = settings['epochs']
        self.window = settings['window_size']
        ## End code

    def generate_training_data(self, corpus):
        """
        Generate training data from the given corpus.
        This function processes the input corpus to create training examples for the Word2Vec model.
        - It first counts the occurrences of each word in the corpus.
        - Then, it creates a vocabulary of unique words and assigns each word a unique index.
        - Finally, it generates training pairs consisting of target words and their surrounding context words.
        """
        # ## Start code
        # word_counts = np.unique(corpus)
        # self.words_list = word_counts
        self.words_list = list(set(corpus))
        self.v_count = len(self.words_list)
        #self.v_count = len(word_counts)
        self.word_index ={word: i for i, word in enumerate(self.words_list)}
        self.index_word = {i: word for i, word in enumerate(self.words_list)}
        #training_data = corpus
        training_data = []
        for i, word in enumerate(corpus):
            start = max(0, i - self.window)
            end = min(len(corpus), i + self.window + 1)

            for j in range(start, end):
                if j != i:
                    training_data.append((word, corpus[j]))
        return training_data
        ## End code

    def word2onehot(self, word):
        """
        Convert a word into a one-hot encoded vector.
        Output:
        - A one-hot vector of length equal to the vocabulary size.
        """
        ## Start code
        word_vec = np.zeros(self.v_count)
        word_index = self.word_index[word]
        word_vec[word_index] = 1
        return word_vec
        ## End code

    def train(self, training_data):
        """
        Train the model using the given training data.
        This function initializes the weight matrices and performs forward and backward propagation.
        - Initializes weight matrices w1 (input to hidden) and w2 (hidden to output)
        - Iterates through training data and performs forward pass
        - Computes the error and updates weights using backpropagation
        - Tracks and prints loss for each epoch
        """
        ## Start code
        self.w1 = np.random.uniform(-1,1,(self.v_count,self.n))
        self.w2 = np.random.uniform(-1,1,(self.n,self.v_count))
        losses = []
        for i in range(self.epochs):
            self.loss = 0
            for target_word, context_word in training_data:
                x = self.word2onehot(target_word)
                y_true = self.word2onehot(context_word)
                #y_true[self.word_index[w]] = 1
                y_pred, h, u = self.forward_pass(x)
                error = y_true - y_pred
                self.backprop(error, h, x)
                # self.loss += -np.sum([u[word] for word in training_data[i] if word in self.word_index])
                # self.loss += np.sum([u[word] for word in training_data[i] if word in self.word_index])
                #self.loss += -np.sum(y_true * np.log(y_pred))
                epsilon = 1e-10
                self.loss -= np.sum(y_true * np.log(y_pred + epsilon))
            if i % 5 == 0:
              print('Epoch:', i, "Loss:", self.loss)
        ## End code

    def softmax(self, x):
        """
        Apply softmax function.
        This function normalizes the input values into probabilities, ensuring that they sum to 1.
        - It exponentiates each value in x to ensure non-negativity.
        - It divides each exponentiated value by the sum of all exponentiated values to normalize them into a probability distribution.
        Output:
        - A probability distribution where the sum of all elements equals 1.
        """
        ## Start code
        # e_x = np.exp(x)/np.sum(np.exp(x))
        # return e_x
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum()

        ## End code

    def forward_pass(self, x):
        """
        Forward pass through the network.
        This function takes a one-hot encoded word vector as input and performs the following steps:
        - Computes the hidden layer by multiplying the input vector with the first weight matrix.
        - Computes the output layer values by multiplying the hidden layer with the second weight matrix.
        - Applies the softmax function to get the probability distribution over the vocabulary.
        Output:
        - The predicted probability distribution (y_c), hidden layer activations (h), and raw scores before softmax (u).
        """
        ## Start code
        # self.h = np.dot(self.W.T,X).reshape(self.N,1)
        # h = np.dot(self.w1.T,x)
        # u = np.dot(self.w2.T,h)
        h = np.dot(x, self.w1)
        u = np.dot(h, self.w2)
        y_c = self.softmax(u)
        return y_c, h, u
        ## End code

    def backprop(self, e, h, x):
        """
        Backpropagation step to update weights.
        This function updates the weight matrices using gradient descent.
        - Computes the gradient of the loss with respect to the second weight matrix (w2).
        - Computes the gradient of the loss with respect to the first weight matrix (w1).
        - Updates w1 and w2 using the learning rate and computed gradients.
        """
        ## Start code
        dl_dw2 = np.outer(h,e)
        #dl_dw1 = np.outer(x,np.dot(self.w2,e))
        dl_dw1 = np.outer(x, np.dot(self.w2, e))
        # self.w1 += self.lr * dl_dw1
        # self.w2 += self.lr * dl_dw2
        self.w2 -= self.lr * dl_dw2
        self.w1 -= self.lr * dl_dw1
        ## End code

    def word_vec(self, word):
        """
        Retrieve the word vector for a given word.
        """
        ## Start code
        # v_w = self.w1[self.word_index[word]]
        # return v_w
        if word in self.word_index:
            return self.w1[self.word_index[word]]
        else:
            return None
        ## End code
    def vec_sim(self, word, top_n):
        """
        Find top N most similar words based on cosine similarity,
        excluding the word itself.
        """
        if word not in self.word_index:
            return []

        word_idx = self.word_index[word]
        target_vector = self.w1[word_idx]

        dot_products = np.dot(self.w1, target_vector)
        word_norms = np.linalg.norm(self.w1, axis=1)
        target_norm = np.linalg.norm(target_vector)
        similarities = dot_products / (word_norms * target_norm)

        # Set similarity with itself to -1 so it won't be in top N
        similarities[word_idx] = -1

        top_indices = np.argsort(similarities)[::-1][:top_n]
        return [(self.index_word[idx], float(similarities[idx])) for idx in top_indices]


        # v_w1 = self.word_vec(word)
        # word_vector_norm = norm(v_w1)
        # all_norms = np.array([norm(self.w1[i]) for i in range(self.v_count)])
        # # word_sim = np.dot(self.w1,v_w1)
        # # word_sim /= np.linalg.norm(self.w1, axis=1) * np.linalg.norm(v_w1)
        # # words_sorted = np.argsort(-word_sim)
        # similarities = []
        # for i in range(self.v_count):
        #     #v_w2 = self.w1[i]
        #     cos_sim = np.dot(v_w1, self.w1[i]) / (word_vector_norm * all_norms[i])
        #     similarities.append((self.index_word[i], cos_sim))

        ## End code
        #return [(self.index_word[i], word_sim[i]) for i in words_sorted[:top_n]]
        # return sorted(similarities, key=lambda x: x[1], reverse=True)[:top_n]


In [86]:
text = "Natural language processing and machine learning open up fascinating possibilities, allowing machines to analyze,\
 understand, and respond to human language in ways that were once thought impossible."

In [91]:
corpus = [word.lower() for word in text.split()]

# tokens = [' ', '.', ',',"?"]
# for i in range (len(corpus[0])):
#   if corpus[0][i][-1] in tokens:
#     corpus[0][i] = corpus[0][i][:-1]

w2v = word2vec(settings)

training_data = w2v.generate_training_data(corpus)

w2v.train(training_data)

Epoch: 0 Loss: 399.58849521914567
Epoch: 5 Loss: 514.2201209836763
Epoch: 10 Loss: 858.3782907378849
Epoch: 15 Loss: 1655.2511233078117
Epoch: 20 Loss: 2212.6405590357963
Epoch: 25 Loss: 2290.113983751027
Epoch: 30 Loss: 2300.7763996859135
Epoch: 35 Loss: 2285.8998097167223
Epoch: 40 Loss: 2279.559242063803
Epoch: 45 Loss: 2279.559242063803


In [93]:
word = "machine"
vec = w2v.word_vec(word)
print(word, vec)

# Find similar words
w2v.vec_sim("machine", 5)

machine [ 33.61616145 -42.0168299  -55.81369974 -13.77658166 -11.48039286
   3.0925903   -6.399108   -33.32873671   4.62688482 -24.98989089]


[('fascinating', 0.9787328739701204),
 ('were', 0.9610179561940602),
 ('that', 0.9436386448237125),
 ('impossible.', 0.9314480889784025),
 ('language', 0.8981565681780966)]